### Use FIXED max_length, not fill with 0

In [1]:
import torch
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import trange
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor
from functools import wraps, partial
from torch.nn.modules.sparse import Embedding
from torch.optim import Adam, SGD
import torch.nn as nn
from tqdm import tqdm
from random_utils import set_seed
import pandas as pd

In [2]:
# model_name = "loi_combine_arc_obqa.pkl"
model_name='google-t5/t5-large'
# "loi_with_padding_1.pkl"#
# model_name = (
#     "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
# )
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-large")

model_original = T5ForConditionalGeneration.from_pretrained(
    model_name, device_map=f"auto")  # 'auto')
model = model_original
# model.hf_device_map

/home/ldh0033@auburn.edu/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
DEVICE = 0

In [4]:

DATABASE_NAME = ['obqa_fact','arc_easy','arc_hard']#,'race','mctest',]
dataset_test=[]
dataset_train=[]
[dataset_test.append([]) for x in DATABASE_NAME]
moi_index=[]

for i, dataname in enumerate(DATABASE_NAME):
    dataset_test[i]=pickle.load(
        open(f"multiple_choice_datasets/{dataname}_test.pkl", "rb"))
    moi_index.append(len(dataset_train))
    dataset_train.extend(pickle.load(
        open(f"multiple_choice_datasets/{dataname}_train.pkl", "rb")))
for i in range(2):    
    print(f""" max: {max([len(x[i]) for x in dataset_train])} 
        min:{min([len(x[i]) for x in dataset_train])}
            avg:{sum([len(x[i]) for x in dataset_train])/len(dataset_train)}
            len: {len(dataset_train)}""")
BATCH_SIZE=10

 max: 857 
        min:66
            avg:213.01621232136424
            len: 8327
 max: 153 
        min:1
            avg:20.48372763300108
            len: 8327


In [5]:
# import textwrap

# ques=4
# print(textwrap.fill(dataset_test[ques][0]))
# print('correct: ',dataset_test[ques][1],'length',len(dataset_test[0][0]))


In [6]:
def hook(hook_before, oldfunc, hook_after):

    def foo(*args, **kwargs):
        hook_before(*args, **kwargs)
        aa = oldfunc(*args, **kwargs)
        hook_after(*args, **kwargs)
        return aa

    return foo

In [7]:
def new_compute_bias(self, query_length, key_length, device=None):
    """Compute binned relative position bias"""
    if device is None:
        device = self.relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[
        :, None
    ]
    memory_position = torch.arange(
        key_length, dtype=torch.long, device=device)[None, :]

    relative_position = (
        memory_position - context_position
    )  # shape (query_length, key_length)
    if self.is_decoder:
        values = values.permute([2, 0, 1]).unsqueeze(
            0
        )  # shape (1, num_heads, query_length, key_length)
        return values

    anchors = self.anchor_array
    values = []
    for anchor in anchors:
        mot = [[anchor[idx], anchor[idx+1]] for idx in range(len(anchor)-1)]
        max_answer_length =300 #max([x[1] - x[0] for x in mot])
        # print(a, b, c, d, max_answer_length)
        context_position_new = context_position.clone()
        for i in range(1, len(mot)):
            context_position_new[mot[i][0]: mot[i][1]
                                 ] = context_position_new[mot[i][0]: mot[i][1]] - mot[0][0]
        context_position_new[-1] = mot[0][0] + 2 * max_answer_length
        memory_position_new = context_position_new.clone().view(1, -1)
        relative_position = (
            memory_position_new - context_position_new
        )  # shape (query_length, key_length)
        for i in range(len(mot)):
            for j in range(len(mot)):
                if i != j:
                    x = mot[i]
                    y = mot[j]
                    relative_position[x[0]: x[1], y[0]: y[1]] += max_answer_length
        relative_position_bucket = self._relative_position_bucket(
            relative_position,  # shape (query_length, key_length)
            bidirectional=(not self.is_decoder),
            num_buckets=self.relative_attention_num_buckets,
            max_distance=self.relative_attention_max_distance,
        )
        value = self.relative_attention_bias(relative_position_bucket)
        values.append(value)
    values = torch.stack(values)  # shape [1, 91, 91, 16]
    values = values.permute(
        [0, 3, 1, 2]
    )  # shape (batch size, num_heads, query_length, key_length)
    return values

In [8]:

import textwrap


extra_dim_learning = []


def set_mode(MODE):
    itself = model.encoder.block[0].layer[0].SelfAttention
    if MODE == "new":
        # itself.forward = partial(modified_self_attention_forward, itself)
        itself.compute_bias = partial(new_compute_bias, itself)
        model.forward = hook(input_before_hooker, partial(
            T5ForConditionalGeneration.forward, model), input_after_hooker)

    else:
        # itself.forward = partial(
        #     model.encoder.block[0].layer[0].SelfAttention.__class__.forward, itself
        # )
        itself.compute_bias = partial(
            model.encoder.block[0].layer[0].SelfAttention.__class__.compute_bias, itself
        )
        model.forward = T5ForConditionalGeneration.forward


def check_encoded(all_input_ids):
    anchors = []
    for input_ids in all_input_ids:
        # print('\n'.join([f'{x.item()},{y}' for x,y in zip(input_ids, tokens)][50:]))
        original = input_ids.tolist()
        anchor = []
        for i in range(len(input_ids)):
            if (
                i < len(input_ids) - 2
                and input_ids[i] == 41
                and input_ids[i + 1] == 3
                and input_ids[i + 2] == 61
            ) or original[i] == 1:
                anchor.append(i)
        anchors.append(anchor)
    return anchors


def input_before_hooker(*args, **kwargs):
    input_ids = kwargs["input_ids"]
    # print('old ',input_ids)
    anchors = check_encoded(input_ids)
    # final_inputs = []
    # for input_id, anchor in zip(input_ids, anchors):
    #     input_id = input_id.tolist()
        
    #     real_max_len = max([anchor[idx+1] - anchor[idx]
    #                        for idx in range(len(anchor)-1)])
    #     max_answer_length=real_max_len
    #     for x in reversed(range(1, len(anchor))):
    #         if anchor[x] - anchor[x - 1] < max_answer_length:
    #             [
    #                 input_id.insert(anchor[x], 0)
    #                 for _ in range(max_answer_length - (anchor[x] - anchor[x - 1]))
    #             ]

    #     final_inputs.append(input_id)

    # max_length = max([len(input) for input in final_inputs])
    # mask = [[1]*max_length]*len(final_inputs)
    # for idx, input in enumerate(final_inputs):
    #     for x in range(max_length):
    #         if x >= len(input):
    #             mask[idx][x] = 0
    #     for x in range(max_length-len(input)):
    #         input.append(0)
    # kwargs["input_ids"] = torch.tensor(final_inputs).to(input_ids.device)
    # kwargs['attention_mask'] = torch.tensor(mask).to(input_ids.device)
    # # print('new ',kwargs["input_ids"])
    # anchors = check_encoded(kwargs["input_ids"])
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = anchors


def input_after_hooker(*args, **kwargs):
    model.encoder.block[0].layer[0].SelfAttention.anchor_array = None


print(textwrap.fill(dataset_train[0][0]))

# set_mode('old')
set_mode('new')
# run_model([dataset_train[0][0],dataset_train[1][0]])
# run_model(["""A person wants to start""", 'mot hai ba'])

the sun is the source of energy for physical cycles on Earth. The sun
is responsible for \n ( ) puppies learning new tricks ( ) children
growing up and getting old ( ) flowers wilting in a vase ( ) plants
sprouting, blooming and wilting


In [9]:

def measure_unalike(arr, print_arr=False):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    if print_arr:
        print(arr)
    return 1 - ((arr / n)**2).sum()


question_to_do = 5
per_question = 20


def get_model_forward(input_ids, attention_mask, model=model):
    with torch.no_grad():
        start = []
        [start.append([0]) for x in range(len(input_ids))]
        for k in range(100):
            # print(torch.tensor(start).shape)
            result = model(
                input_ids=input_ids.to(DEVICE),
                attention_mask=attention_mask.to(DEVICE),
                decoder_input_ids=torch.tensor(start).to(DEVICE),
                output_attentions=True,
            )
            item = result.logits.argmax(dim=2)[:, -1]
            # print('loi',result.logits.shape, item)
            for index in range(len(item)):
                start[index].append(item[index].item())
            if torch.allclose(item, torch.tensor(1)):
                break
            #     break
    result = []
    for batch in start:
        y = -1
        for index, x in enumerate(batch):
            if x == 1:
                y = index
                break
        result.append(batch[:y+1] if y > -1 else batch)
    return [tokenizer.decode(x, skip_special_tokens=True) for x in result]


def run_model(input_strs):
    if input_strs is str:
        input_strs = [input_strs]
    input_ids_wrapper = tokenizer(
        input_strs, padding=True, return_tensors='pt')

    answer = get_model_forward(input_ids_wrapper['input_ids'],
                               input_ids_wrapper['attention_mask'])
    return answer

In [10]:
kk = [(index, x, y) for index, (x, y) in enumerate(model.named_parameters())
      if y.requires_grad == True]
[(index, x) for index, x, y in kk if "decoder" in x]
len(kk)
all_position_weight = [
    y for index, x, y in kk if ("extra_dimension_embedding" in x) or (
        ("encoder" in x) and ("relative_attention_bias" in x))
]
to_train_model = [y for index, x, y in kk]

In [11]:
data_array = [
    (ques, answer, ques.split(" ( ) ")[1:])
    for ques, answer in [
        (
            dataset_train[x][0],
            dataset_train[x][1],
        )
        for x in range(len(dataset_train))
    ]
]

In [12]:
# [(print(idx),CheckTransform.__call__(None, data_array[idx])) for idx,x in enumerate(data_array)]

In [13]:
class CheckTransform(object):

    def __call__(self, sample):
        # print(f"'{sample[1]}'")
        try:
            return {
            "input_ids": sample[0],
            "label_index": sample[2].index(sample[1]),
            "all_labels": sample[2],
            }
        except:
            raise Exception('cao')
            print('all answer: ',sample[2])
            print('answer',sample[1])

class CustomDataset(Dataset):

    def __init__(self, dataset_array, transform=None):
        self.dataset = dataset_array
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.transform(self.dataset[idx])


def collate(datas):
    wrapper = tokenizer([x["all_labels"][x['label_index']] for x in datas],
                        padding=True)
    wrapper["label_ids"] = torch.tensor(wrapper.pop("input_ids"))
    # wrapper["label_index"] = torch.tensor([x["label_index"] for x in datas])
    for k in wrapper["label_ids"]:
        k[k == tokenizer.pad_token_id] = -100
    wrapper["all_decoder_attention_masks"] = torch.tensor(
        wrapper.pop("attention_mask"))

    for_input = tokenizer([x["input_ids"] for x in datas], padding=True)
    wrapper['input_ids'] = torch.tensor(for_input.pop('input_ids'))
    wrapper['attention_mask'] = torch.tensor(for_input.pop('attention_mask'))
    return wrapper


loi_dataloader = DataLoader(
    CustomDataset(
        data_array,
        CheckTransform(),
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate,
)
# for k in loi_dataloader:
#     print(k["all_label_ids"])
#     break

In [14]:
from transformers.optimization import Adafactor, AdafactorSchedule

# attention 898704
# hidden state 242688
# classification_layer = nn.Linear(242688, 4).to(DEVICE)
# optimizer =AdamW(lr=1e-4)
optimizer = Adafactor(
    to_train_model,  # + [x for x in classification_layer.parameters()],
    relative_step=False,
    lr=1e-4,
)

In [15]:
def turn_position_learning(on):
    for x in all_position_weight:
        x.requires_grad = on


loss_running_score = 0
correct_running_score = 0
conform_running_score = 0
count = 0
extra_info = ""
res_tokens = []
accumulate = 10
optimizer.zero_grad()
set_seed(42)
turn_position = False
turn_position_learning(False)
for learn_pos in range(5):
    pbar = tqdm(loi_dataloader)
    for wrapper in pbar:
        count += 1
        result = model(
            input_ids=wrapper["input_ids"].to(DEVICE),
            attention_mask=wrapper['attention_mask'].to(DEVICE),
            labels=wrapper['label_ids'].to(DEVICE),
            decoder_attention_mask=wrapper["all_decoder_attention_masks"].to(
                DEVICE),  # output_attentions=True
        )
        loss = result.loss
        loss_running_score = loss_running_score * 0.9 + loss.item() * 0.1
        if loss != 0:
            loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        with torch.no_grad():
            if count % 100 == 0:
                extra_info = run_model(dataset_test[0][0][0])
            pbar.set_description_str(f"Loss: {loss_running_score:.3f}")
            pbar.set_postfix_str(extra_info)
    before_lr = optimizer.param_groups[0]["lr"]
    after_lr = optimizer.param_groups[0]["lr"]
    print("Epoch %d: SGD lr %.4f -> %.4f" % (learn_pos, before_lr, after_lr))

  0%|                                                                               | 0/833 [00:00<?, ?it/s]

Loss: 0.901:   0%|                                                                  | 0/833 [00:03<?, ?it/s]

Loss: 0.901:   0%|                                                                  | 0/833 [00:03<?, ?it/s]

Loss: 0.901:   0%|                                                          | 1/833 [00:03<45:00,  3.25s/it]

Loss: 1.565:   0%|                                                          | 1/833 [00:03<45:00,  3.25s/it]

...

Loss: 0.004: 100%|█████████████████████▉| 832/833 [07:34<00:00,  1.53it/s, ['buy less with monopoly money']]

Loss: 0.004: 100%|█████████████████████▉| 832/833 [07:34<00:00,  1.53it/s, ['buy less with monopoly money']]

Loss: 0.004: 100%|██████████████████████| 833/833 [07:34<00:00,  1.58it/s, ['buy less with monopoly money']]

Loss: 0.004: 100%|██████████████████████| 833/833 [07:34<00:00,  1.83it/s, ['buy less with monopoly money']]

Epoch 4: SGD lr 0.0001 -> 0.0001


In [16]:
model.save_pretrained("loi_combine_model.pkl", from_pt=True)

### Measure accuracy and answer coverage

In [17]:
ll=1
wrong_answers=[[] for x in range(ll)]
got_2=[[] for x in range(ll)]
got_1=[[] for x in range(ll)]
answers=[[] for x in range(ll)]
for ix in range(ll):
    print(f'Name {DATABASE_NAME[ix]}')
    #[pickle.load(open(f"multiple_choice_datasets/obqa_fact_test.pkl", "rb"))]:
    # print(f"test {data==dataset_test}")
    count = 0
    count1 = 0
    count2 = 0
    count10 = 0
    total = 0
    data=dataset_test[ix]
    pbar1 = trange(len(data))
    for ques in pbar1:
        question = data[ques][0]
        key = data[ques][1]
        total += 1
        answer = run_model(question)[0]
        answers[ix].append(answer)
        if key == answer:
            count += 1
        else:
            wrong_answers[ix].append(ques)
        if key[0] == answer[0]:
            count1 += 1
            got_1[ix].append(ques)
        if key[:2] == answer[:2]:
            count2 += 1
            got_2[ix].append(ques)
        if answer in question:
            count10 += 1
        pbar1.set_postfix_str(
            f"{count}, {count1}, {count2}, {count10},{total}")

Name obqa_fact


  0%|                                                                               | 0/500 [00:00<?, ?it/s]

  0%|                                                                 | 0/500 [00:00<?, ?it/s, 0, 0, 0, 1,1]

  0%|                                                         | 1/500 [00:00<07:18,  1.14it/s, 0, 0, 0, 1,1]

  0%|                                                         | 1/500 [00:01<07:18,  1.14it/s, 1, 1, 1, 2,2]

...

100%|████████████████████████████████████████████▊| 498/500 [02:46<00:00,  2.38it/s, 290, 351, 345, 491,499]

100%|████████████████████████████████████████████▉| 499/500 [02:46<00:00,  2.42it/s, 290, 351, 345, 491,499]

100%|████████████████████████████████████████████▉| 499/500 [02:46<00:00,  2.42it/s, 291, 352, 346, 492,500]

100%|█████████████████████████████████████████████| 500/500 [02:46<00:00,  3.09it/s, 291, 352, 346, 492,500]

100%|█████████████████████████████████████████████| 500/500 [02:46<00:00,  3.00it/s, 291, 352, 346, 492,500]

In [18]:
idx=[x for x in got_2[0] if x  in wrong_answers[0]]
for ix in idx:
    print(textwrap.fill(dataset_test[0][ix][0]),'\nAnswer:', answers[0][ix],'\nCorrect:',dataset_test[0][ix][1])

hawks eat lizards. A red-tailed hawk is searching for prey. It is most
likely to swoop down on \n ( ) an eagle ( ) a cow ( ) a gecko ( ) a
deer 
Answer: a cow 
Correct: a gecko
friction is used for stopping a vehicle by brakes. which of these
would stop a car quicker? \n ( ) a wheel with wet brake pads ( ) a
wheel without brake pads ( ) a wheel with worn brake pads ( ) a wheel
with dry brake pads 
Answer: a wheel with worn brake pads 
Correct: a wheel with dry brake pads
hypothesis means scientific guess about the cause and effect of an
event. Which of these is a hypothesis? \n ( ) The ice caps will
completely melt if global warming continues ( ) The earth is round ( )
The earth revolves around the sun ( ) Gravity causes objects to fall 
Answer: The earth revolves around the sun 
Correct: The ice caps will completely melt if global warming continues
studying a soil sample means studying the microorganisms in that soil.
They studied the soil by using \n ( ) plants ( ) a telescope ( ) ro

### Measure resilient

In [19]:
def measure_unalike(arr):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    return 1 - ((arr / n)**2).sum()


measure_unalike(["a", "a", "a"])

0.0

In [20]:
# for data in [dataset_test]:
#     count = 0
#     count1 = 0
#     count2 = 0
#     count10 = 0
#     total = 0
#     question_index = range(5)
#     pbar1 = tqdm(question_index)
#     unalike = []
#     for ques1 in pbar1:
#         answer_set = []
#         for m in trange(24):
#             ques = ques1 * 24 + m
#             question = data[ques][0]
#             key = data[ques][1]
#             question_convert = check(question)
#             if question_convert is None:
#                 continue
#             total += 1
#             answer, _, _, _ = get_model_forward(question_convert.to(DEVICE),
#                                                 model=model2)
#             answer_set.append(answer)
#         unalike.append(measure_unalike(answer_set))
# print(f"Mean unalikeability: {sum(unalike)/len(unalike)}")

In [21]:
# pbar = trange(0, len(dataset_train), 24)
# loss_score = 0
# count = 0
# extra_info = ""
# set_seed(42)
# res_tokens=[]
# for learn_pos in range(10):
#     for step in pbar:
#         count += 1
#         # if count>20:
#         #     break
#         # print(textwrap.fill(dataset_train[0][0]))
#         input_tokens = check(dataset_train[step][0])
#         if input_tokens is None:
#             continue
#         labels = tokenizer.encode(dataset_train[step][1], return_tensors="pt")
#         result = model(input_ids=input_tokens.to(DEVICE), labels=shape(labels).to(DEVICE))

#         optimizer.zero_grad()
#         loss =loss_fn(result.logits[0][learn_pos],labels[0][learn_pos].to(DEVICE))
#         loss_score = loss_score * 0.9 + loss.item() * 0.1
#         if loss.item()!=0:
#             loss.backward()
#         optimizer.step()
#         # scheduler.step()
#         with torch.no_grad():
#             if count % 10 == 0:
#                 extra_info, res_tokens = get_model_forward(check(dataset_test[0][0]).to(DEVICE))
#             pbar.set_description_str(f"Loss: {loss_score:.2f}")
#             pbar.set_postfix_str(res_tokens[:learn_pos+2])
# pass